In [133]:
# generate data
import Distributions
using Random

Random.seed!(42)

n_samples = 100
dimensionality = 2
rθ = [-0.9, 0.5]
inputs = randn(n_samples, dimensionality)
σ(w, x) = 1/(1+exp(-w'x))
πs = [σ(rθ, inputs[i, :]) for i in 1:n_samples]
outputs = [rand(Distributions.Bernoulli(πs[i])) for i in 1:n_samples]

100-element Vector{Bool}:
 0
 1
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 1
 ⋮
 0
 0
 0
 1
 1
 1
 1
 0
 0
 1
 1
 0

In [134]:
πs

100-element Vector{Float64}:
 0.6511341085084169
 0.5955877066136283
 0.41569281403288216
 0.4401202616771189
 0.0948478615047964
 0.7980167159032031
 0.5925325939573203
 0.4297422164226539
 0.9010022425685328
 0.27875157627555786
 0.3567134898728285
 0.6348073825555255
 0.35346337423728474
 ⋮
 0.24237193452009162
 0.5967636594091411
 0.35118819613650737
 0.8995973133106453
 0.8071131893153088
 0.8281026193563655
 0.946293396612311
 0.4445554826173346
 0.030654537619569756
 0.7083517955565383
 0.6905272043138534
 0.664220963467057

In [135]:
using Turing

In [146]:
logistic(w,x) = 1/(1+exp(-w'x))

@model user_preferences(inputs, outputs, σ²; D=2, K=2) = begin
    
    n_samples = length(outputs)
    weights   = Vector{Vector}(undef, n_samples)
    
    # State sequence.
    s = tzeros(Int, n_samples)

    m = Vector(undef, K)

    # Transition matrix.
    T = Vector{Vector}(undef, K)
    for i in 1:K
        T[i] ~ Dirichlet(ones(K)/K)
        m[i] ~ Normal(i, 0.5)
    end
    
    θ ~ MvNormal(zeros(D), sqrt(σ²)) 
    
    s[1] ~ Categorical(K)
    y[1] ~ Normal(m[s[1]], 0.1)
    
    for i in 1:n_samples
        weights[i] ~ MvNormal(inputs[i, :], sqrt(10.0))
    end

    for i in 1:n_samples
        π = logistic(weights[i], θ)
        outputs[i] ~ Bernoulli(π)
    end
end;

In [147]:
# Settings of the Hamiltonian Monte Carlo (HMC) sampler.
iterations = 2000
ϵ = 0.05
τ = 10

chains = sample(user_preferences(inputs, outputs, 0.1), HMC(ϵ, τ), iterations)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:53


Chains MCMC chain (2000×211×1 Array{Float64, 3}):

Iterations        = 1:2000
Thinning interval = 1
Number of chains  = 1
Samples per chain = 2000
Wall duration     = 57.9 seconds
Compute duration  = 57.9 seconds
parameters        = weights[23][2], weights[71][1], weights[13][1], weights[50][2], θ[1], weights[64][2], weights[16][1], weights[94][2], weights[54][2], weights[61][1], weights[10][1], weights[100][2], weights[3][1], weights[36][1], weights[48][2], weights[68][2], weights[26][1], weights[97][2], weights[22][1], weights[73][1], weights[95][2], weights[47][1], weights[50][1], weights[86][2], weights[94][1], weights[8][1], weights[35][2], weights[21][1], weights[63][1], weights[70][2], weights[23][1], weights[79][1], weights[52][1], weights[14][2], weights[43][2], weights[67][1], weights[85][2], weights[37][1], weights[91][1], weights[55][2], weights[27][1], weights[7][2], weights[55][1], weights[59][1], weights[63][2], weights[81][2], weights[84][2], weights[53][1], weights[17]

In [143]:
d = 2

sumstats = summarize(chains, mean, std)
mθ, vθ   = sumstats.nt.mean[1:2], sumstats.nt.std[1:2]
mw       = [sumstats.nt.mean[i:i+d-1] for i in d+1:d:d*n_samples+d]

100-element Vector{Vector{Float64}}:
 [-0.3707910707212212, 0.10978605027468057]
 [-0.6446666563145693, 0.06717351737636983]
 [-0.25525664403644993, -0.4512387887354433]
 [-0.10749051898140538, -1.2292644445478855]
 [1.9248749644648147, -1.3448797286421503]
 [-1.2899085176062282, 0.9202588989138918]
 [-0.7085719764023278, -0.08443717833487417]
 [-0.20936265676234958, -0.09501313409301644]
 [-2.7531792125640666, -0.18630473757698693]
 [1.2793799926371148, -0.06325146360812399]
 [0.6676239456599308, 0.8426771642600577]
 [0.02439750162156193, 1.4961241621475294]
 [0.1597748525973603, -0.1590707063080306]
 ⋮
 [0.823484460352815, -1.0666512819434288]
 [0.6795790419771863, 1.2806736975993103]
 [0.440946461022478, -0.8640322692545783]
 [-1.6402924538516876, 1.4604938242156038]
 [-1.1688977937419558, 1.3804005630694323]
 [-0.9787819220013382, 1.6242655471971208]
 [-2.130502370652669, 2.2312911097091113]
 [0.3238102979171723, -0.3767963371931768]
 [3.2506938920253976, -1.340027464418827]
 [-0.8

In [144]:
e_outputs = round.([logistic(mθ, mw[i]) for i in 1:n_samples])

100-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 0.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0

In [145]:
sum(outputs .== e_outputs)

81